環境構築

In [0]:
%matplotlib inline

In [12]:
from google.colab import files
upload = files.upload()

Saving sudoku.png to sudoku (2).png


In [13]:
from google.colab import files
upload = files.upload()

Saving white50.png to white50 (2).png


In [14]:
!apt install tesseract-ocr
!apt install libtesseract-dev
!pip install pyocr
!curl -L https://github.com/tesseract-ocr/tessdata/raw/master/jpn.traineddata > jpn.traineddata
!cp jpn.traineddata /usr/share/tesseract-ocr/tessdata

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libtesseract-dev is already the newest version (4.00~git2288-10f4998a-2).
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   145  100   145    0     0    656      0 --:--:-- --:--:-- --:--:--   656
100 34.0M  100 34.0M    0     0  22.9M      0  0:00:01  0:00:01 --:--:-- 56.8M


In [0]:
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import sys
import pyocr
import pyocr.builders
from PIL import Image, ImageDraw, ImageFont

In [16]:
#OCRが使用可能かをチェック
tools = pyocr.get_available_tools()
if len(tools) == 0:
    print("OCR tool is not found")
    sys.exit(1)

#OCRツール名を表示
tool = tools[0]
print("OCR tool is '%s'" % (tool.get_name()))

#OCR対応言語を表示
langs = tool.get_available_languages()
print("Available languages: %s" % ", ".join(langs))
lang = langs[0]
print("Will use lang '%s'" % (lang))

OCR tool is 'Tesseract (sh)'
Available languages: osd, eng
Will use lang 'osd'


In [19]:
img = cv2.imread("sudoku.png")

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

cnts = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[1]

for pt in cnts:
  x, y, w, h = cv2.boundingRect(pt)

imgC = img[x+25:h-25, y+25:w-25]
imgDATA = cv2.resize(imgC, (900, 900))

#code = {}
sudokudata = []

for y in range(9):
  for x in range(9):
    coordinate = (str(y + 1) + str(x + 1))


    #ここが変数になる
    contoursY1 = y * 100
    contoursY2 = contoursY1 + 100
    contoursX1 = x * 100
    contoursX2 = contoursX1 + 100
    #ここが幅のパラメーター
    contours = 20
    
    filename = "img" + coordinate


    image = imgDATA[contoursY1 + 5 + contours:contoursY2 + 5 - contours,
                    contoursX1 + 2 + contours:contoursX2 + 2 - contours]


    #二値化
    image = cv2.threshold(image,180,255,cv2.THRESH_BINARY)[1] 
    
    #判定に入る
    checkimg = image[15:35, 15:35]
    checkimg = checkimg.flatten()
    if 0 not in checkimg:
      sudokudata.append(0)
    else:
      #print("number")
      #まだ完成形ではないため、数字をpngで出力して余白を追加させる
      cv2.imwrite("img{}.png".format(filename), image)
      
      newimg = Image.open("img{}.png".format(filename))
      imgwhite = Image.open('white50.png')
      imgwhite.paste(newimg, (25, 25))
      imgwhite.save("img{}a.png".format(filename), quality=95)

      txt = tool.image_to_string(
          Image.open("img{}a.png".format(filename)),
          lang="eng",
         builder=pyocr.builders.TextBuilder(tesseract_layout=6))
      
      sudokudata.append(int(txt))
      
    
    
    #code[filename] = image
print(sudokudata)

[0, 0, 3, 2, 0, 0, 8, 0, 0, 0, 2, 0, 0, 1, 9, 0, 0, 7, 9, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 5, 1, 0, 0, 6, 0, 0, 0, 9, 0, 0, 6, 5, 0, 0, 3, 0, 0, 7, 9, 0, 0, 4, 0, 0, 0, 0, 8, 5, 0, 0, 9, 0, 0, 5, 0, 0, 0, 0, 0, 0, 6, 0, 0, 7, 0, 0, 9, 3, 0, 0, 1]
